# Distribution in Practice

### Introduction

In the last lesson, we learned about the various distribution strategies in a redshift database.  Let's do a quick review.

As we saw, the **key** distribution is generally the most preferable choice, as it ensures a co-locating of data when performing our joins.

> Key distribution

<img src="./dist_key_full.jpg" width="90%">

Sometimes, however a key distribution can lead to skewed data, in which case the **all** strategy -- which copies over a dimension table to each node -- can be used.  Finally, if the dimension table is over say 2 - 3 million rows, the **even** distribution, which performs round robin can be used.  However, as we know, this can result in shuffling.

<img src="./exported.jpg" width="100%">

Now how do we determine which distribution will be used in our cluster?  Well, as we'll we see, we make this determination when creating our tables.

### Implementing an All Strategy

Let's say that are working with our movie rentals database.

<img src="./starred_films.png" width="50%">

We would expect to have fewer movie directors than movies themselves -- with a single director creating multiple movies.  So let's say that we wish to use an **all** distribution strategy to copy movie directors to all nodes.  We can do so with the following:

In [44]:
create_directors_query = """CREATE TABLE IF NOT EXISTS "directors" (
    "id" integer NOT NULL DEFAULT nextval('directors_id_seq'),
    "name" VARCHAR(200),
    "movie_id" INTEGER
) diststyle all;"""

In [45]:
cursor.execute(create_directors_query)

In [46]:
conn.commit()

So we can see that the **all** distribution is placed on the table itself.

### Implementing an Even Strategy

Let's take another look at our star schema.  Here, it likely makes sense to perform `keydist` with the `release date id` -- both on the movies table, and the `release_dates` table.

> We like this strategy, because with the **key** distribution, we can place our release dates on the node that has movies from that date.

<img src="./starred_films.png" width="50%">

In addition, we should probably sort our tables by date. Let's see how we can do so.

In [47]:
create_movies_query = """CREATE TABLE IF NOT EXISTS "movies" (
    "id" integer NOT NULL DEFAULT nextval('directors_id_seq'),
    "title" VARCHAR(200),
    "runtime" INTEGER,
    "release_date_id" INTEGER sortkey distkey,
    "popularity" INTEGER
);"""

create_release_dates = """CREATE TABLE "release_dates" (
    "id" integer NOT NULL DEFAULT nextval('release_dates_id_seq') distkey,
    "release_date" DATETIME sortkey,
    "year" INTEGER,
    "month" INTEGER,
    "day" INTEGER
);"""

In [48]:
cursor.execute(create_movies_query)

In [49]:
conn.commit()

So here, we can see that we specify the `sortkey` as the `release_date`, which sorts the data into those values, and provides for easier scanning of records through the data blocks.  And we also set a distribution strategy of key both on the `release_date` and the `release_date_id`. 

So notice that here, the distribution strategy on the column that determines the distribution of the data.

### Final Thoughts

One thing to note is that the fact table can only have one `distkey` per fact table.  And because all our dimension tables should point to the fact table, this means that we only have one `distkey` on a single dimension table.  

The `distall` strategy, however, can be used on multiple tables.  And can be used for any dimension table that is not too large.

### Resources

[AWS Best Dist Key](https://docs.aws.amazon.com/redshift/latest/dg/c_best-practices-best-dist-key.html)

[AWS Distribution Examples](https://docs.aws.amazon.com/redshift/latest/dg/c_Distribution_examples.html)

[Redshift Deep Dive](https://www.youtube.com/watch?t=578&v=iuQgZDs-W7A&feature=youtu.be&ab_channel=AWSOnlineTechTalks)
